In [118]:
# !pip install protobuf==3.20.3
# !pip install --upgrade tokenizers
# !pip install crewai crewai_tools langchain_community python-dotenv
# !pip install exifread
# !pip install chardet

In [80]:
# from google.colab import userdata
import os
from dotenv import load_dotenv
load_dotenv()

True

In [135]:
#use the below directories for local runtime
input_dir = "example-files/original-pics/"  # Replace with your input folder
downscaled_dir_for_web = "example-files/downscaled-pics/"  # Replace with your output folder
downscaled_dir_for_LLM = "example-files/super-downscaled-pics/"
selected_dir = "example-files/selected-pics/"  # Replace with your output folder
example_web_files = "example-files/example-markdown/"
watermark_filepath = "example-files/logo_watermark-youtube.png"
img_filter_threshold = 7.5

In [ ]:
from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder, preserving date taken metadata.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    # exif_data = img.getexif()  # Get EXIF data
                    # date_taken = exif_data.get(36867) if exif_data else None  # Extract date taken

                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality

                    # Preserve EXIF data in the resized image
                    # if date_taken:
                    resized_img.info['exif'] = img.info['exif']

                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir_for_web, max_dimension=1080)
resize_images(input_dir, downscaled_dir_for_LLM, max_dimension=512)    #use lower res to save on LLM tokens

Resized P4060050.jpg and saved to example-files/downscaled-pics/
Resized P4060054.jpg and saved to example-files/downscaled-pics/
Resized P4060213-1.jpg and saved to example-files/downscaled-pics/
Resized P4060269.jpg and saved to example-files/downscaled-pics/
Resized P4060293.jpg and saved to example-files/downscaled-pics/
Resized P4060300.jpg and saved to example-files/downscaled-pics/
Resized P4060396.jpg and saved to example-files/downscaled-pics/
Resized P4060410.jpg and saved to example-files/downscaled-pics/
Resized P4060448.jpg and saved to example-files/downscaled-pics/
Resized P4060478.jpg and saved to example-files/downscaled-pics/
Resized P4060487.jpg and saved to example-files/downscaled-pics/
Resized P4060495.jpg and saved to example-files/downscaled-pics/
Resized P4060506.jpg and saved to example-files/downscaled-pics/
Resized P4060523.jpg and saved to example-files/downscaled-pics/
Resized P4060050.jpg and saved to example-files/super-downscaled-pics/
Resized P4060054.

In [139]:
# import base64
import os
import imghdr
import exifread
from pydantic import BaseModel, Field
from typing import List, Optional

class ImageObject(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    filepath_full: str = Field(..., description="Complete filepath for the image.")
    date_taken: Optional[str] = Field(None, description="Date the image was originally captured.")
    # base64_encoding: Optional[str] = Field(None, description="Base64 encoded string of the image file.")
    # base64 no longer needed with OAI vision tool
    description: Optional[str] = Field(None, description="Objective description of what is in the image.")
    assessment: Optional[str] = Field(None, description="Qualitative assessment of the image.")
    quality_score: Optional[float] = Field(None, ge=1, le=10, description="Quality score of the image based on assessment.")
    title: Optional[str] = Field(None, description="Title of the image.")
    summary: Optional[str] = Field(None, description="Publication ready and web friendly summary of the image.")
    

def images_to_objects(input_folder: str) -> List[ImageObject]:
    """
    Converts images in the input folder to a list of ImageObject objects
    with base64 encoded values and EXIF data.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A list of ImageObject objects with base64 encoded strings and EXIF data.
    """

    image_objects = []

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    # encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    
                    # Extract EXIF data
                    image_file.seek(0)  # Reset file pointer to the beginning
                    tags = exifread.process_file(image_file, stop_tag="EXIF DateTimeOriginal")
                    date_taken = tags.get("EXIF DateTimeOriginal")
                    date_only = date_taken.values.split(" ")[0] if date_taken else None

                    image_object = ImageObject(
                        filename=filename,
                        filepath_full=image_path,
                        # base64_encoding=encoded_string,
                        date_taken=date_only
                    )
                    image_objects.append(image_object)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_objects

# Example usage
# downscaled_dir = "path/to/your/downscaled_dir"
image_objects = images_to_objects(downscaled_dir_for_LLM)

In [140]:
# Print the details of each image object
for obj in image_objects:
    print(f"Filename: {obj.filename}")
    print(f"Filepath: {obj.filepath_full}")
    print(f"Description: {obj.description}")
    print(f"Assessment: {obj.assessment}")
    print(f"Quality Score: {obj.quality_score}")
    print(f"Date Taken: {obj.date_taken}")
    print(f"Title: {obj.title}")
    print(f"Summary: {obj.summary}")
    # print(f"Base64 Encoding (first 40 chars): {obj.base64_encoding[:40] if obj.base64_encoding else 'None'}")
    print("-" * 20)

Filename: P4060050.jpg
Filepath: example-files/super-downscaled-pics/P4060050.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
--------------------
Filename: P4060054.jpg
Filepath: example-files/super-downscaled-pics/P4060054.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
--------------------
Filename: P4060213-1.jpg
Filepath: example-files/super-downscaled-pics/P4060213-1.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
--------------------
Filename: P4060269.jpg
Filepath: example-files/super-downscaled-pics/P4060269.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
--------------------
Filename: P4060293.jpg
Filepath: example-files/super-downscaled-pics/P4060293.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: N

In [141]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import VisionTool

llm_art_critic = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.5,
    api_key = os.getenv('GEMINI_API_KEY')
)

In [142]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
                          as a practicing artist. In your youth, you explored a wide range of artistic mediums,
                          from painting and sculpture to photography and performance art, giving you a deep understanding
                          of the creative process. Your passion for art has led you to travel the world,
                          immersing yourself in diverse cultures and artistic traditions. You possess a profound
                          knowledge of art history, from ancient cave paintings to contemporary installations,
                          and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
                          Your critical approach is characterized by a keen eye for visual storytelling,
                          an appreciation for the artist's process, and an openness to experimentation.
                          You are a master of articulating your thoughts in a clear and engaging manner,
                          providing insightful critiques that are both objective and constructive."""

art_critic_agent = Agent(
    llm=llm_art_critic,
    role="Art Critic",
    goal="Analyse visual art and write compelling SEO content",
    backstory=agent_critic_background,
    verbose=True,
    memory=True,
    # output_pydantic=ImageAnalysisResult
)

In [143]:

                        # - Your assessment will be based on how good you believe others will find the art work. If
                        #     you yourself like a picture but you think others don't find it striking or interesting
                        #     the image will get a lower score.

agent_critic_task = Task(
    description="""You will provide a detailed and objective analysis of the images given to you as input.
                        Your analysis have to include:
                      1. Objective Description:
                        - Provide a neutral and factual description of the image's subject matter, composition,
                            and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
                        - Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
                        - Technical Execution: Analyze the artist's technical skill in handling the chosen medium
                            and their mastery of fundamental artistic principles (e.g., perspective, proportion,
                            light and shadow).
                        - Composition and Design: Evaluate the effectiveness of the composition in guiding the
                            viewer's eye and creating visual interest. Consider the use of balance, rhythm,
                            and focal points.
                        - Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
                            and their impact on the overall mood and atmosphere.
                        - Conceptual Exploration: If applicable, delve into the conceptual underpinnings of the work.
                            Identify any symbolism, metaphors, or narratives that contribute to its meaning.
                        - Originality and Innovation: Comment on the originality of the work and whether it
                            demonstrates a unique approach or expands upon existing artistic conventions.¨
                      3. Overall Impression:
                        - Summarize your overall impression of the artwork, highlighting its strengths and weaknesses.
                        - Offer constructive suggestions for improvement, if applicable.
                        - If an image is not good, boring, or poor, you will say so.
                      4. Objective Quality Score:
                        - Assign a score between 1 and 10 (10 being the highest) based on your 
                            assessment of the criteria above.
                        - It is crucial that your score is free from personal bias or preference. Focus on the
                            artwork's technical merits, compositional strength, and conceptual depth.
                        - If an image is not good, boring, or poor, you will say so.
                        - You don't give a good score to boring images. You do give a good score to intriguing images.

                      Remember: Your role is to provide an objective and insightful critique, without aiming to please
                          your assessment is attachedd to your reputation, so you focus on being objective.
                          If an image is not good, boring, or poor, you are not afraid to say so.
                          
                      Here is everything you need to know about the image:
                      {crew_image_input}
                      """,
    expected_output="""A list containing the image name, description, critical assessment, and quality score.""",
    agent=art_critic_agent,
    asynchronous=False,
    tools=[VisionTool()],
    output_pydantic=ImageObject #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

# Here is additional information on the image: {crew_image_input}


In [144]:
critique_crew = Crew(
    agents=[art_critic_agent
            ],
    tasks=[
           agent_critic_task,
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_critique_log.txt"
)


2025-01-04 17:45:35,581 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [145]:
# results = critique_crew.kickoff(inputs={ "image_url": image_objects[0].base64_encoding})

# crew_image_input = { 
#     "image_path_url": "example-files/downscaled-pics/P4060448.jpg"
#     }

# results = critique_crew.kickoff(inputs=crew_image_input)

In [146]:
crew_image_input = []

for image in image_objects:
    # image.base64_encoding = "sverd"
    crew_image_input.append({
        # "image_url": image.base64_encoding,
        "crew_image_input": image,
    })

# crew_image_input = image_objects

# crew_image_input[0]

In [148]:
reviewed_images = critique_crew.kickoff_for_each(inputs=crew_image_input)
print(reviewed_images)

2025-01-04 17:46:29,186 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
- Technical Execution: Analyze the artist's technical skill in handling the chosen medium
    and their mastery of fundamental artistic principles (e.g., perspective, proportion,
    light and shadow).
- Composition and Design: Evaluate the effectiveness of the composition in guiding the
    viewer's eye and creating visual interest. Consider the use of balance, rhythm,
    and focal points.
- Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
    and th

2025-01-04 17:46:40,082 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Art Critic
## Final Answer: 
{
  "filename": "P4060050.jpg",
  "filepath_full": "example-files/super-downscaled-pics/P4060050.jpg",
  "date_taken": "2023:04:06",
  "description": "A photograph depicting a desert landscape. The foreground features a single, large cactus. The background showcases a range of mountains under a clear sky. The overall color palette is muted earth tones, with browns, tans, and blues dominating. The image is sharply focused, with good detail visible in both the foreground and background elements. The composition is fairly simple, with a clear distinction between the foreground and background elements. The lighting suggests it was taken during the daytime.",
  "assessment": "Objective Description:\nThe image is a landscape photograph showing a single, large cactus in the foreground and a range of mountains under a clear sky in the background. The color palette is primarily muted earth tones, with browns, tans, and blues. The focus is sharp, with deta

2025-01-04 17:46:54,197 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

 Failed to convert text into a pydantic model due to the following error: litellm.BadRequestError: VertexAIException BadRequestError - {
  "error": {
    "code": 400,
    "message": "Invalid JSON payload received. Unknown name \"default\" at 'tools[0].function_declarations[0].parameters.properties[2].value': Cannot find field.\nInvalid JSON payload received. Unknown name \"default\" at 'tools[0].function_declarations[0].parameters.properties[3].value': Cannot find field.\nInvalid JSON payload received. Unknown name \"default\" at 'tools[0].function_declarations[0].parameters.properties[4].value': Cannot find field.\nInvalid JSON payload received. Unknown name \"default\" at 'tools[0].function_declarations[0].parameters.properties[5].value': Cannot find field.\nInvalid JSON payload received. Unknown name \"default\" at 'tools[0].functio

2025-01-04 17:49:03,845 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Art Critic
## Final Answer: 
{
  "filename": "P4060495.jpg",
  "filepath_full": "example-files/super-downscaled-pics/P4060495.jpg",
  "date_taken": "2023:04:06",
  "description": "A photograph depicting a saddled horse standing in a desert landscape. The background features mountains and sparse vegetation under a clear sky. The lighting suggests either early morning or late afternoon.",
  "assessment": "Objective Description:\nThe image is a photograph of a horse in a desert setting. The horse is centrally positioned, taking up a significant portion of the frame. It appears to be a light brown or tan color, and is saddled. The background is a vast, open desert landscape with muted colors, featuring mountains in the distance and sparse, low-lying vegetation. The sky is clear and bright, suggesting daytime. The overall composition is fairly simple and straightforward, with a clear focal point on the horse.\n\nCritical Assessment:\nTechnical Execution: The technical execution i

2025-01-04 17:49:20,733 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Art Critic
## Final Answer: 
{
  "filename": "P4060506.jpg",
  "filepath_full": "example-files/super-downscaled-pics/P4060506.jpg",
  "date_taken": "2023:04:06",
  "description": "A photograph depicting a view from horseback. The foreground shows the mane and ears of the horse, while another horse is visible in the background. The setting appears to be a desert or arid landscape, with mountains in the distance and a setting sun casting warm light. Some sparse desert vegetation is also visible.",
  "assessment": "Objective Description:\nThe image is a photograph taken from a horseback rider's perspective. The composition is straightforward, focusing on the view ahead.  The foreground is dominated by the horse's mane and ears, leading the eye towards the background. The background features another horse and a landscape characterized by mountains under a setting sun. The color palette is warm, dominated by oranges, yellows, and browns, typical of a desert sunset. The texture is

In [149]:
reviewed_images 

[CrewOutput(raw='{\n  "filename": "P4060050.jpg",\n  "filepath_full": "example-files/super-downscaled-pics/P4060050.jpg",\n  "date_taken": "2023:04:06",\n  "description": "A photograph depicting a desert landscape. The foreground features a single, large cactus. The background showcases a range of mountains under a clear sky. The overall color palette is muted earth tones, with browns, tans, and blues dominating. The image is sharply focused, with good detail visible in both the foreground and background elements. The composition is fairly simple, with a clear distinction between the foreground and background elements. The lighting suggests it was taken during the daytime.",\n  "assessment": "Objective Description:\\nThe image is a landscape photograph showing a single, large cactus in the foreground and a range of mountains under a clear sky in the background. The color palette is primarily muted earth tones, with browns, tans, and blues. The focus is sharp, with detail visible in bot

In [ ]:
#consider: save reviews to txt?
#consider: implement averaging of quality score?
#move images above threshold into selected dir
#i move images in and out of the folder
#filter the array to only include the files (filename) that are now in selected-dir
#watermark images
#organize into subfolder and create index.md
#kick off next crew

In [132]:
# Assuming `results` is a list of CrewOutput objects
# filtered_results = [result for result in results if result.pydantic.quality_score > 6]
# filtered_results

In [ ]:
#move images above threshold into selected dir

import shutil

def move_high_quality_images(results, downscaled_dir, selected_dir, threshold=5):
    """Moves images with a quality score over the threshold to selected_dir.

    Args:
        results: The list of CrewOutput objects.
        downscaled_dir: The directory containing downscaled images.
        selected_dir: The directory to move selected images to.
        threshold: The quality score threshold for selecting images.
    """
    os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

    for result in results:
        if result.pydantic and result.pydantic.quality_score > threshold:
            filename = result.pydantic.filename
            source_path = os.path.join(downscaled_dir, filename)
            destination_path = os.path.join(selected_dir, filename)

            if os.path.exists(source_path):
                shutil.move(source_path, destination_path)
                print(f"Moved {filename} to {selected_dir}")
            else:
                print(f"Warning: {filename} not found in {downscaled_dir}")

# Example usage
move_high_quality_images(reviewed_images, downscaled_dir_for_web, selected_dir)

Moved P4060213-1.jpg to example-files/selected-pics/
Moved P4060396.jpg to example-files/selected-pics/
Moved P4060410.jpg to example-files/selected-pics/
Moved P4060448.jpg to example-files/selected-pics/
Moved P4060478.jpg to example-files/selected-pics/
Moved P4060506.jpg to example-files/selected-pics/
Moved P4060523.jpg to example-files/selected-pics/


## if you want to make changes to selected pics, do so here before running the next code block

In [ ]:
#filter the array to only include the files (filename) that are now in selected-dir

def filter_results_in_selected_dir(results, selected_dir):
    """Filters the list to only contain image objects currently in selected_dir.

    Args:
        results: The list of CrewOutput objects.
        selected_dir: The directory containing selected images.

    Returns:
        A filtered list containing only image objects in selected_dir.
    """
    selected_filenames = set(os.listdir(selected_dir))
    filtered_results = [result for result in results if result.pydantic and result.pydantic.filename in selected_filenames]
    return filtered_results

filtered_results = filter_results_in_selected_dir(reviewed_images, selected_dir)
filtered_results

In [159]:
# from PIL import Image

def watermark_images(selected_dir, watermark_filepath):
    watermark = Image.open(watermark_filepath)
    for filename in os.listdir(selected_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(selected_dir, filename)
            img = Image.open(filepath)
            width, height = img.size
            shortest_dim = min(width, height)
            watermark_width = int(0.18 * shortest_dim)
            watermark_height = int((watermark_width / watermark.width) * watermark.height)
            watermark = watermark.resize((watermark_width, watermark_height))
            padding = int(0.01 * shortest_dim)
            img.paste(watermark, (padding, height - watermark_height - padding), watermark)
            img.save(filepath)

watermark_images(selected_dir, watermark_filepath)

In [160]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w", encoding="utf-8") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                # f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

organize_images(selected_dir)

Moved and renamed 'P4060293.jpg' to 'example-files/selected-pics/P4060293\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060293'
Moved and renamed 'P4060478.jpg' to 'example-files/selected-pics/P4060478\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060478'
Moved and renamed 'P4060523.jpg' to 'example-files/selected-pics/P4060523\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060523'


In [161]:
subfolder_inputs = []
for subfolder_name in os.listdir(selected_dir):
        # Assuming subfolder names match image filenames without extension
    image_object = next((obj for obj in image_objects if obj.filename.startswith(subfolder_name)), None)
    if image_object:
        #TIS IS WHERE IT STOPS!
        subfolder_inputs.append({
            "subfolder_name": selected_dir + subfolder_name,
            "image_object": image_object  # Pass the relevant image object
        })

subfolder_inputs

[{'subfolder_name': 'example-files/selected-pics/P4060293',
  'image_object': ImageObject(filename='P4060293.jpg', filepath_full='example-files/super-downscaled-pics/P4060293.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)},
 {'subfolder_name': 'example-files/selected-pics/P4060478',
  'image_object': ImageObject(filename='P4060478.jpg', filepath_full='example-files/super-downscaled-pics/P4060478.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)},
 {'subfolder_name': 'example-files/selected-pics/P4060523',
  'image_object': ImageObject(filename='P4060523.jpg', filepath_full='example-files/super-downscaled-pics/P4060523.jpg', date_taken='2023:04:06', description=None, assessment=None, quality_score=None, title=None, summary=None)}]

In [163]:
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    # max_tokens=4096,
    api_key = os.environ['GEMINI_API_KEY']
)

In [164]:
# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [165]:
read_examples_task = Task(
    description="""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [166]:

write_image_web_content = Task(
    description="""You are provided with information and analysis of an image.
                   Each object you receive contains the following information about an image:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Read the content of this list and store it in your memory.
                   For each image object, generate:
                   1. A compelling title based on the description and assessment.
                   2. A compelling summary incorporating key features and artistic qualities.

                   You make sure the title and summary you write are in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   Store the generated title and summary within the image object,
                   by updating the 'title' and 'summary' attributes.

                   You do not need to open any files. The necessary information has been provided to 
                   you as task input right here:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description="""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.
                   
                   You make sure the title and summary are based on the task input:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=art_critic_agent,
    asynchronous=False,
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [167]:

process_all_subfolders_task = Task(
    description="""Process a single subfolder using provided input data
                   Navigate to the directory named in '{current_sub_folder}'.
                   Perform the processes:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the subfolder data list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the image objects:
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   It is important that you do not rewrite the image summary or the image title. You should use the
                   summary and the title you received from the Art Critic Agent""",
    expected_output="""'index.md' files updated/created with content from image objects.
                    AND SAVED IN THE SUBFOLDER IN: '{current_sub_folder}'.
                    Characters aligned with UTF-8 standards and best practices.
                    the file must include the attributes:
                    "title: "
                    "authors:
                      -admin"
                    "tags:
                      - Photography"
                    "image:
                      placement: 2
                      preview_only: false"
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
)


In [168]:
full_crew = Crew(
    agents=[art_critic_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_log.txt"
)


2025-01-04 18:10:54,683 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [240]:
# consider adding task guardrails to verify output

In [170]:
crew_inputs = []

for sub_folder in subfolder_inputs:
    crew_inputs.append({
        "current_sub_folder": sub_folder,
        "example_web_files": example_web_files
    })

results = full_crew.kickoff_for_each(inputs=crew_inputs)
# print(results)

2025-01-04 18:11:50,186 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1.md
- example-files/example-markdown/index2.md
- example-files/example-markdown/index3.md


# Agent: Web Developer
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"example-files/example-markdown/index1.md\"}"
## Tool Output: 
---
title: 'Roadrunner in Big Bend National Park'
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  focal_point: ""
  preview_only: false
---

A Greater Roadrunner stands alert on a rocky outcrop in Big Bend National Park, Texas. The rugged mountains provide a stunning backdrop to 

2025-01-04 18:13:57,894 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
---
title: "Fiery Saguaros at Sunset"
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  preview_only: false
---

Bathed in the warm glow of a Big Bend sunset, this image captures a moment of serene power. A lone saguaro cactus stands sentinel against a fiery sky, its silhouette stark against the intense colors. The interplay of light and shadow reveals the intricate textures of the desert landscape, creating a scene both dramatic and peaceful.  The rich hues and compelling composition draw the viewer into the heart of the desert.


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
F

2025-01-04 18:15:39,681 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
---\ntitle: "Desert Bloom: April Light"\nauthors:\n  - admin\ntags:\n  - Photography\ndate: "2023-04-06T00:00:00Z"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nBathed in the soft light of an April day, this desert landscape unfolds with quiet grace.  The composition subtly guides the viewer's eye across the scene, revealing a delicate balance between the harsh beauty of the desert and the promise of springtime renewal.  A sense of stillness and vastness pervades the image, inviting contemplation and wonder.


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1

In [144]:
import chardet

def ensure_utf8_encoding(selected_dir):
    """
    Recursively processes all Markdown files in the selected directory and
    its subdirectories, ensuring they are encoded in UTF-8.
    
    Args:
        selected_dir (str): Path to the folder containing Markdown files.
    """
    for root, dirs, files in os.walk(selected_dir):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Read the file and detect its current encoding
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    detected = chardet.detect(raw_data)
                    current_encoding = detected['encoding']
                
                # If not UTF-8, convert and overwrite
                if current_encoding.lower() != 'utf-8':
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(raw_data.decode(current_encoding))
                    print(f"Converted {file_path} to UTF-8.")
                else:
                    print(f"{file_path} is already UTF-8.")

# Example usage
ensure_utf8_encoding(selected_dir)

Converted example-files/selected-pics/P4060478\index.md to UTF-8.
Converted example-files/selected-pics/P4070608\index.md to UTF-8.
Converted example-files/selected-pics/P5270188\index.md to UTF-8.
Converted example-files/selected-pics/P6040181\index.md to UTF-8.
Converted example-files/selected-pics/P6220060\index.md to UTF-8.
Converted example-files/selected-pics/P6220221\index.md to UTF-8.
Converted example-files/selected-pics/P6220309\index.md to UTF-8.
Converted example-files/selected-pics/P6230571\index.md to UTF-8.
Converted example-files/selected-pics/P6230622\index.md to UTF-8.
Converted example-files/selected-pics/P7230316\index.md to UTF-8.
Converted example-files/selected-pics/P8020068-2\index.md to UTF-8.
Converted example-files/selected-pics/P8090407\index.md to UTF-8.


### code graveyard for reference